In [2]:
# ch05 逻辑回归
# 步骤1：创建Spark会话对象
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('log_reg').getOrCreate()

In [3]:
# 步骤2:读取数据集
df = spark.read.csv("Log_Reg_dataset.csv", inferSchema=True,header=True)

In [4]:
# 步骤3: 探究式数据分析
print(df.count(), len(df.columns))

20000 6


In [5]:
df.printSchema()

root
 |-- Country: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Repeat_Visitor: integer (nullable = true)
 |-- Platform: string (nullable = true)
 |-- Web_pages_viewed: integer (nullable = true)
 |-- Status: integer (nullable = true)



In [6]:
df.show(5)

+---------+---+--------------+--------+----------------+------+
|  Country|Age|Repeat_Visitor|Platform|Web_pages_viewed|Status|
+---------+---+--------------+--------+----------------+------+
|    India| 41|             1|   Yahoo|              21|     1|
|   Brazil| 28|             1|   Yahoo|               5|     0|
|   Brazil| 40|             0|  Google|               3|     0|
|Indonesia| 31|             1|    Bing|              15|     1|
| Malaysia| 32|             0|  Google|              15|     1|
+---------+---+--------------+--------+----------------+------+
only showing top 5 rows



In [7]:
df.describe().show()

+-------+--------+-----------------+-----------------+--------+-----------------+------------------+
|summary| Country|              Age|   Repeat_Visitor|Platform| Web_pages_viewed|            Status|
+-------+--------+-----------------+-----------------+--------+-----------------+------------------+
|  count|   20000|            20000|            20000|   20000|            20000|             20000|
|   mean|    null|         28.53955|           0.5029|    null|           9.5533|               0.5|
| stddev|    null|7.888912950773227|0.500004090187782|    null|6.073903499824976|0.5000125004687693|
|    min|  Brazil|               17|                0|    Bing|                1|                 0|
|    max|Malaysia|              111|                1|   Yahoo|               29|                 1|
+-------+--------+-----------------+-----------------+--------+-----------------+------------------+



In [8]:
df.groupBy('Country').count().show()

+---------+-----+
|  Country|count|
+---------+-----+
| Malaysia| 1218|
|    India| 4018|
|Indonesia|12178|
|   Brazil| 2586|
+---------+-----+



In [9]:
df.groupBy('Platform').count().show()

+--------+-----+
|Platform|count|
+--------+-----+
|   Yahoo| 9859|
|    Bing| 4360|
|  Google| 5781|
+--------+-----+



In [10]:
df.groupBy('Status').count().show()

+------+-----+
|Status|count|
+------+-----+
|     1|10000|
|     0|10000|
+------+-----+



In [11]:
df.groupBy('Country').mean().show()

+---------+------------------+-------------------+---------------------+--------------------+
|  Country|          avg(Age)|avg(Repeat_Visitor)|avg(Web_pages_viewed)|         avg(Status)|
+---------+------------------+-------------------+---------------------+--------------------+
| Malaysia|27.792282430213465| 0.5730706075533661|   11.192118226600986|  0.6568144499178982|
|    India|27.976854156296664| 0.5433051269288203|   10.727227476356397|  0.6212045793927327|
|Indonesia| 28.43159796354081| 0.5207751683363442|    9.985711939563148|  0.5422893742814913|
|   Brazil|30.274168600154677|  0.322892498066512|    4.921113689095128|0.038669760247486466|
+---------+------------------+-------------------+---------------------+--------------------+



In [12]:
df.groupBy('Platform').mean().show()

+--------+------------------+-------------------+---------------------+------------------+
|Platform|          avg(Age)|avg(Repeat_Visitor)|avg(Web_pages_viewed)|       avg(Status)|
+--------+------------------+-------------------+---------------------+------------------+
|   Yahoo|28.569226087838523| 0.5094837204584644|    9.599655137437875|0.5071508266558474|
|    Bing| 28.68394495412844| 0.4720183486238532|    9.114908256880733|0.4559633027522936|
|  Google|28.380038055699707| 0.5149628092025601|    9.804878048780488|0.5210171250648676|
+--------+------------------+-------------------+---------------------+------------------+



In [13]:
df.groupBy('Status').mean().show()

+------+--------+-------------------+---------------------+-----------+
|Status|avg(Age)|avg(Repeat_Visitor)|avg(Web_pages_viewed)|avg(Status)|
+------+--------+-------------------+---------------------+-----------+
|     1| 26.5435|             0.7019|              14.5617|        1.0|
|     0| 30.5356|             0.3039|               4.5449|        0.0|
+------+--------+-------------------+---------------------+-----------+



In [15]:
# 步骤4: 特征工程
# 使用VectorAssembler将类别变量转换成数值形式
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler

In [17]:
search_engine_indexer = StringIndexer(inputCol='Platform', outputCol='Search_Engine_Num').fit(df)
df=search_engine_indexer.transform(df)
df.show(3, False)

+-------+---+--------------+--------+----------------+------+-----------------+
|Country|Age|Repeat_Visitor|Platform|Web_pages_viewed|Status|Search_Engine_Num|
+-------+---+--------------+--------+----------------+------+-----------------+
|India  |41 |1             |Yahoo   |21              |1     |0.0              |
|Brazil |28 |1             |Yahoo   |5               |0     |0.0              |
|Brazil |40 |0             |Google  |3               |0     |1.0              |
+-------+---+--------------+--------+----------------+------+-----------------+
only showing top 3 rows



In [19]:
df.groupBy('Platform').count().orderBy('count', ascending=False).show(5, False)

+--------+-----+
|Platform|count|
+--------+-----+
|Yahoo   |9859 |
|Google  |5781 |
|Bing    |4360 |
+--------+-----+



In [20]:
# 下一步就是将这些值的每一个表示成独热编码位置向量的形式
from pyspark.ml.feature import OneHotEncoder
search_engine_encoder = OneHotEncoder(inputCol = 'Search_Engine_Num', outputCol='Search_Engine_Vector')
df=search_engine_encoder.transform(df)
df.show(3, False)

+-------+---+--------------+--------+----------------+------+-----------------+--------------------+
|Country|Age|Repeat_Visitor|Platform|Web_pages_viewed|Status|Search_Engine_Num|Search_Engine_Vector|
+-------+---+--------------+--------+----------------+------+-----------------+--------------------+
|India  |41 |1             |Yahoo   |21              |1     |0.0              |(2,[0],[1.0])       |
|Brazil |28 |1             |Yahoo   |5               |0     |0.0              |(2,[0],[1.0])       |
|Brazil |40 |0             |Google  |3               |0     |1.0              |(2,[1],[1.0])       |
+-------+---+--------------+--------+----------------+------+-----------------+--------------------+
only showing top 3 rows



In [21]:
df.groupBy('Search_Engine_Vector').count().orderBy('count', ascending=False).show(5,False)

+--------------------+-----+
|Search_Engine_Vector|count|
+--------------------+-----+
|(2,[0],[1.0])       |9859 |
|(2,[1],[1.0])       |5781 |
|(2,[],[])           |4360 |
+--------------------+-----+



In [24]:
df.head()

Row(Country='India', Age=41, Repeat_Visitor=1, Platform='Yahoo', Web_pages_viewed=21, Status=1, Search_Engine_Num=0.0, Search_Engine_Vector=SparseVector(2, {0: 1.0}))

In [26]:
country_indexer = StringIndexer(inputCol='Country', outputCol='Country_Num').fit(df)

In [27]:
df = country_indexer.transform(df)
df.groupBy('Country').count().orderBy('count', ascending=False).show(5, False)

+---------+-----+
|Country  |count|
+---------+-----+
|Indonesia|12178|
|India    |4018 |
|Brazil   |2586 |
|Malaysia |1218 |
+---------+-----+



In [28]:
df.groupBy('Country_Num').count().orderBy('count', ascending=False).show(5, False)

+-----------+-----+
|Country_Num|count|
+-----------+-----+
|0.0        |12178|
|1.0        |4018 |
|2.0        |2586 |
|3.0        |1218 |
+-----------+-----+



In [32]:
country_encoder=OneHotEncoder(inputCol='Country_Num', outputCol='Country_Vector')
df=country_encoder.transform(df)

In [33]:
df.select(['Country', 'Country_Num', 'Country_Vector']).show(3, False)

+-------+-----------+--------------+
|Country|Country_Num|Country_Vector|
+-------+-----------+--------------+
|India  |1.0        |(3,[1],[1.0]) |
|Brazil |2.0        |(3,[2],[1.0]) |
|Brazil |2.0        |(3,[2],[1.0]) |
+-------+-----------+--------------+
only showing top 3 rows



In [34]:
df.groupBy('Country_Vector').count().orderBy('count', ascending=False).show(5, False)

+--------------+-----+
|Country_Vector|count|
+--------------+-----+
|(3,[0],[1.0]) |12178|
|(3,[1],[1.0]) |4018 |
|(3,[2],[1.0]) |2586 |
|(3,[],[])     |1218 |
+--------------+-----+



In [35]:
df_assembler = VectorAssembler(inputCols=['Search_Engine_Vector', 'Country_Vector', 'Age', 'Repeat_Visitor', 'Web_pages_viewed'],
                              outputCol='features')
df = df_assembler.transform(df)
df.printSchema()

root
 |-- Country: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Repeat_Visitor: integer (nullable = true)
 |-- Platform: string (nullable = true)
 |-- Web_pages_viewed: integer (nullable = true)
 |-- Status: integer (nullable = true)
 |-- Search_Engine_Num: double (nullable = false)
 |-- Search_Engine_Vector: vector (nullable = true)
 |-- Country_Num: double (nullable = false)
 |-- Country_Vector: vector (nullable = true)
 |-- features: vector (nullable = true)



In [36]:
df.select(['features', 'Status']).show(10, False)

+-----------------------------------+------+
|features                           |Status|
+-----------------------------------+------+
|[1.0,0.0,0.0,1.0,0.0,41.0,1.0,21.0]|1     |
|[1.0,0.0,0.0,0.0,1.0,28.0,1.0,5.0] |0     |
|(8,[1,4,5,7],[1.0,1.0,40.0,3.0])   |0     |
|(8,[2,5,6,7],[1.0,31.0,1.0,15.0])  |1     |
|(8,[1,5,7],[1.0,32.0,15.0])        |1     |
|(8,[1,4,5,7],[1.0,1.0,32.0,3.0])   |0     |
|(8,[1,4,5,7],[1.0,1.0,32.0,6.0])   |0     |
|(8,[1,2,5,7],[1.0,1.0,27.0,9.0])   |0     |
|(8,[0,2,5,7],[1.0,1.0,32.0,2.0])   |0     |
|(8,[2,5,6,7],[1.0,31.0,1.0,16.0])  |1     |
+-----------------------------------+------+
only showing top 10 rows



In [37]:
model_df = df.select(['features', 'Status'])

In [38]:
# 步骤5:划分数据集
training_df, test_df = model_df.randomSplit([0.75, 0.25])
print(training_df.count())

15073


In [39]:
training_df.groupBy('Status').count().show()

+------+-----+
|Status|count|
+------+-----+
|     1| 7517|
|     0| 7556|
+------+-----+



In [40]:
print(test_df.count())

4927


In [41]:
test_df.groupBy('Status').count().show()

+------+-----+
|Status|count|
+------+-----+
|     1| 2483|
|     0| 2444|
+------+-----+



In [42]:
# 步骤6 构建和训练逻辑回归模型
from pyspark.ml.classification import LogisticRegression
log_reg = LogisticRegression(labelCol='Status').fit(training_df)

In [43]:
# 训练结果
train_results = log_reg.evaluate(training_df).predictions

In [45]:
train_results.filter(train_results['Status'] == 1).filter(train_results['prediction'] ==1).select(['Status', 'prediction', 'probability']).show(10, False)

+------+----------+----------------------------------------+
|Status|prediction|probability                             |
+------+----------+----------------------------------------+
|1     |1.0       |[0.17956564443999434,0.8204343555600057]|
|1     |1.0       |[0.17956564443999434,0.8204343555600057]|
|1     |1.0       |[0.09276450795275529,0.9072354920472447]|
|1     |1.0       |[0.09276450795275529,0.9072354920472447]|
|1     |1.0       |[0.09276450795275529,0.9072354920472447]|
|1     |1.0       |[0.09276450795275529,0.9072354920472447]|
|1     |1.0       |[0.09276450795275529,0.9072354920472447]|
|1     |1.0       |[0.0455909623433493,0.9544090376566506] |
|1     |1.0       |[0.0455909623433493,0.9544090376566506] |
|1     |1.0       |[0.0455909623433493,0.9544090376566506] |
+------+----------+----------------------------------------+
only showing top 10 rows



In [46]:
# 步骤7: 在测试集上评估逻辑回归模型
results=log_reg.evaluate(test_df).predictions
results.printSchema()

root
 |-- features: vector (nullable = true)
 |-- Status: integer (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [47]:
results.select(['Status', 'prediction']).show(10, False)

+------+----------+
|Status|prediction|
+------+----------+
|0     |0.0       |
|0     |0.0       |
|0     |0.0       |
|0     |0.0       |
|0     |0.0       |
|0     |0.0       |
|0     |0.0       |
|1     |0.0       |
|0     |0.0       |
|1     |0.0       |
+------+----------+
only showing top 10 rows



In [48]:
# 混淆矩阵
# 预测真，真实真
tp = results[(results.Status == 1) & (results.prediction == 1)].count()
# 预测真，真实假
fp = results[(results.Status == 0) & (results.prediction == 1)].count()
# 预测假，真实假
tn = results[(results.Status == 0) & (results.prediction == 0)].count()
# 预测假，真实真
fn = results[(results.Status == 1) & (results.prediction == 0)].count()

In [53]:
# 准确率
accracy = float((tp + tn) / (tp + fp + tn + fn))
print(accracy)

0.934645829104932


In [50]:
# 召回率
recall = float(tp) / (tp + fn)

In [51]:
print(recall)

0.9343536045106726


In [52]:
# 精确率
precision = float(tp) / (tp + fp)
print(precision)

0.9358612343686971
